# Example Analysis of Steinmetz Dataset

This notebook demonstrates basic usage of all analysis tools and visualization functions. It serves as a quick-start guide for analyzing the Steinmetz dataset.

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_loader import SteinmetzDataLoader
from neural_analysis import NeuralAnalyzer
from behavior_analysis import BehaviorAnalyzer
from visualization import NeuralViz

# Set plotting style
plt.style.use('seaborn')
sns.set_context("talk")

## 1. Data Loading and Basic Visualization

In [ ]:
# Initialize components
loader = SteinmetzDataLoader()
analyzer = NeuralAnalyzer()
behavior_analyzer = BehaviorAnalyzer()
viz = NeuralViz()

# Load example session
session_data = loader.load_session(11)

# Create raster plot for first neuron
example_spikes = session_data['spikes'][0]
example_trials = np.arange(len(example_spikes))
viz.plot_raster(example_spikes[0], example_trials, (-0.5, 0.5), 
                title='Example Neuron Raster Plot')
plt.show()

## 2. Neural Activity Analysis

In [ ]:
# Compute PSTH
psth, time_bins = analyzer.compute_psth(
    session_data['spikes'][0],
    time_window=(-0.5, 0.5)
)

# Plot PSTH
plt.figure(figsize=(10, 6))
plt.plot(time_bins, psth)
plt.axvline(x=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Time from stimulus onset (s)')
plt.ylabel('Firing rate (Hz)')
plt.title('Peri-Stimulus Time Histogram')
plt.show()

## 3. LFP Analysis Example

In [ ]:
# Compute LFP power spectrum
freqs, power = analyzer.compute_lfp_power(
    session_data['lfp'][:, 0],  # First channel
    freq_range=(1, 100)
)

# Plot power spectrum
plt.figure(figsize=(10, 6))
plt.semilogy(freqs, power)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.title('LFP Power Spectrum')
plt.grid(True)
plt.show()

## 4. Population Analysis Example

In [ ]:
# Prepare population data
time_bins = np.arange(-0.5, 0.5, 0.01)
firing_rates = loader.compute_firing_rates(session_data['spikes'], time_bins)

# Create population activity heatmap
viz.plot_population_activity(firing_rates, time_bins, sort_by='rate')
plt.title('Population Activity (sorted by firing rate)')
plt.show()

## 5. Behavioral Analysis Example

In [ ]:
# Generate example behavioral data
n_trials = len(session_data['spikes'][0])
example_choices = np.random.randint(0, 2, n_trials)
example_outcomes = np.random.randint(0, 2, n_trials)

# Analyze sequential effects
seq_effects = behavior_analyzer.compute_sequential_effects(
    example_choices, 
    example_outcomes
)

# Print results
print("Sequential Effects:")
for key, value in seq_effects.items():
    print(f"{key}: {value:.3f}")

## 6. Cross-Regional Analysis Example

In [ ]:
# Compute coherence between two LFP channels
from scipy import signal

f, Cxy = signal.coherence(
    session_data['lfp'][:, 0],
    session_data['lfp'][:, 1],
    fs=100
)

# Plot coherence
plt.figure(figsize=(10, 6))
plt.plot(f, Cxy)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Coherence')
plt.title('LFP Coherence between Two Channels')
plt.grid(True)
plt.show()

## 7. Interactive Visualization Example

In [ ]:
# Create interactive time series plot of multiple LFP channels
time = np.arange(1000) / 100  # 10 seconds at 100 Hz
lfp_data = session_data['lfp'][:1000, :3]  # First 3 channels
channel_labels = [f'Channel {i+1}' for i in range(3)]

viz.create_interactive_timeseries(time, lfp_data, channel_labels)

## 8. Summary Dashboard

In [ ]:
# Create summary dashboard
summary_data = {
    'PSTH': psth,
    'LFP Power': power,
    'Population Activity': firing_rates[:10],  # First 10 neurons
    'LFP Coherence': Cxy
}

viz.create_summary_plot(summary_data, plot_type='dashboard')
plt.suptitle('Analysis Summary Dashboard', y=1.02)
plt.show()